### Imports 

In [47]:
import model 
import data_proc_methods as dp
from importlib import reload 
import scanpy as sc
import torch
from sklearn.metrics import accuracy_score
from collections import Counter

In [48]:
import numpy as np

In [56]:
from importlib import reload

In [58]:
reload(dp)

<module 'data_proc_methods' from '/Users/fredrikstrom/Documents/KTH Dokument/scRNA_annotation-Thesis-/data_proc_methods.py'>

### Load data 

In [4]:
adata = dp.load_data('subchunk1_1.h5ad')

In [6]:
encoder = dp.get_encoder(adata)

In [7]:
aloader = dp.create_annloader(adata, encoder)

/Users/fredrikstrom/Documents/KTH Dokument/scRNA_annotation-Thesis-/data_proc_methods.py:61: ImplicitModificationWarning: Setting element `.obsm['label']` of view, initializing view as actual.
  adata.obsm['label'] = torch.LongTensor(encoder.transform(adata.obs['supercluster_term']))


In [13]:
bdata = dp.load_data('subchunk1_2.h5ad')

In [6]:
bloader = dp.create_annloader(bdata)

In [60]:
gdata = dp.load_data('nonneuron_jorstad.h5ad', gene_location = 'feature_name')

In [ ]:
fdata,_ = dp.load_data('subchunk1_3.h5ad')

In [ ]:
floader = dp.create_annloader(fdata)

In [49]:
skippidibapp = model.BasicNeuralNetwork(adata.var['Gene'].values,adata.obs['supercluster_term'].cat.categories, layer_dims = [512, 1024, 1024, 1024, 512], lr = 0.0025)

In [9]:
nn_model = model.ResidualNeuralNetwork(adata.var['Gene'].values,adata.obs['supercluster_term'].cat.categories, layer_dims = [512, 512, 512], lr = 0.0025)

### Train model (multiple chunks from brain) 

In [43]:
model.train(aloader,nn_model,label_type='sc',epochs=9)

KeyboardInterrupt: 

In [50]:
model.train(aloader,skippidibapp,label_type='sc',epochs=9)

/Users/fredrikstrom/anaconda3/envs/scRNA/lib/python3.11/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch:1, Loss:0.1148
Epoch:2, Loss:0.0172
Epoch:3, Loss:0.0092
Epoch:4, Loss:0.0062
Epoch:5, Loss:0.0006
Epoch:6, Loss:0.0014
Epoch:7, Loss:0.0205
Epoch:8, Loss:0.0005
Epoch:9, Loss:0.0002


In [ ]:
model.train_nn(bloader,encoder,nn_model,label_type='sc',epochs=5)

In [ ]:
model.train_nn(floader,encoder,nn_model,label_type='sc',epochs=5)

### Align test data and test the models accuracy 

In [24]:
cdata = dp.load_data('Thalamic_Complex_RNA.h5ad')

In [25]:
map = dp.map_genes(adata.var['Gene'].values, cdata.var['Gene'].values)

In [62]:
map_2 = dp.map_genes(adata.var['Gene'].values, gdata.var['feature_name'].values)

In [63]:
map_2[0:20,0:20].toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.

In [26]:
cdata.obsm['aligned'] = dp.align(map,cdata.X)

/var/folders/6k/n1rxygz93j770tq1p431m1kw0000gn/T/ipykernel_72094/4016871610.py:1: ImplicitModificationWarning: Setting element `.obsm['aligned']` of view, initializing view as actual.
  cdata.obsm['aligned'] = dp.align(map,cdata.X)


In [64]:
gdata.obsm['aligned'] = dp.align(map_2,gdata.X)

/var/folders/6k/n1rxygz93j770tq1p431m1kw0000gn/T/ipykernel_72094/47279449.py:1: ImplicitModificationWarning: Setting element `.obsm['aligned']` of view, initializing view as actual.
  gdata.obsm['aligned'] = dp.align(map_2,gdata.X)


In [ ]:
g_pred = encoder.inverse_transform(skippidibapp.forward(torch.FloatTensor(gdata.obsm['aligned'].todense())).detach().argmax(dim=1).numpy())

In [51]:
b_pred = encoder.inverse_transform(skippidibapp.forward(torch.FloatTensor(bdata.X.todense())).detach().argmax(dim=1).numpy())

In [53]:
c_pred = encoder.inverse_transform(skippidibapp.forward(torch.FloatTensor(cdata.obsm['aligned'].todense())).detach().argmax(dim=1).numpy())

/Users/fredrikstrom/anaconda3/envs/scRNA/lib/python3.11/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [54]:
accuracy_score(y_pred=c_pred,y_true=cdata.obs['supercluster_term'].values)

0.9670613404153295

In [52]:
accuracy_score(y_pred=b_pred,y_true=bdata.obs['supercluster_term'].values)

0.9917131663302862

In [ ]:
accuracy_score(y_pred=g_pred,y_true=gdata.obs['supercluster_term'].values)